## Bayes' Theorem Calculator

Whenever you are using a test to determine if something is happening or not, it is time to use Bayes' theorem to avoid comitting the base rate fallacy. However, the mathematical notation and formula may be intimidating. You can use the following calculator to learn how Bayes' theorem works, and to answer practical questions about probability.

Run the following box by clicking on it and pressing `Shift+Return`.

In [1]:
from base_rate import *

display(HTML(style_html))
Javascript(filename='base_rate.js')
Javascript(js)

<IPython.core.display.Javascript object>